In [24]:
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import seaborn as sns
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

In [27]:
# USLGB항구 데이터 로드
df_USLGB = pd.read_excel("USDET.xlsx", encoding='utf-8')
df_USLGB.head()
df_USLGB.shape

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,W/F,D/FEE,합계
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,4350,30000,3288310
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,4350,30000,3312340
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,4350,30000,3303970
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,4350,30000,3724240
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,4350,30000,3484993


(610, 13)

### 유가 선물 내역 데이터랑 합쳐보기

In [28]:
# 유가 데이터 불러오기
df_oil = pd.read_csv("WTI유 선물 내역.csv", encoding='cp949')
df_oil.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2014-01-01,98.70,98.61,98.76,98.54,-,0.28%
1,2014-01-02,95.44,98.50,98.97,95.34,251.25K,-3.30%
2,2014-01-03,93.96,95.47,95.74,93.86,215.52K,-1.55%
3,2014-01-06,93.43,94.18,94.59,93.20,189.38K,-0.56%
4,2014-01-07,93.67,93.60,94.22,93.35,177.18K,0.26%


In [29]:
# 유가 데이터 병합
df_add_oil = pd.merge(df_USLGB, df_oil, left_on="ONBOARD", right_on="날짜", how="left")
df_add_oil.head()

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,W/F,D/FEE,합계,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,4350,30000,3288310,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,4350,30000,3312340,2014-02-18,102.43,100.32,103.25,100.23,177.37K,1.83%
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,4350,30000,3303970,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,4350,30000,3724240,2014-03-17,98.08,99.39,99.39,97.37,227.39K,-0.82%
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,4350,30000,3484993,2014-04-01,99.74,101.53,101.57,99.28,237.56K,-1.81%


In [30]:
# 결측값 확인
df_add_oil.isnull().sum()

ONBOARD          0
VESSEL           0
POL              0
NATION           0
DESTINATION      0
20`              0
RATE             0
EX-RATE          0
O/FRT KRW        0
THC              0
W/F              0
D/FEE            0
합계               0
날짜             141
종가             141
오픈             141
고가             141
저가             141
거래량            141
변동 %           141
dtype: int64

### 소비자 지수 PCE 

In [31]:
# 조인을 위함 월 칼럼 기존 데이터프레임에 추가
df_add_oil['월'] = '0'
for i in range(len(df_add_oil)):
    df_add_oil['월'][i] = str(df_add_oil['ONBOARD'][i])[:7]

In [32]:
df_add_oil.head()

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,...,D/FEE,합계,날짜,종가,오픈,고가,저가,거래량,변동 %,월
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,...,30000,3288310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,...,30000,3312340,2014-02-18,102.43,100.32,103.25,100.23,177.37K,1.83%,2014-02
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,...,30000,3303970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,...,30000,3724240,2014-03-17,98.08,99.39,99.39,97.37,227.39K,-0.82%,2014-03
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,...,30000,3484993,2014-04-01,99.74,101.53,101.57,99.28,237.56K,-1.81%,2014-04


In [33]:
# 미국 구매자지수 불러오기
df_pce = pd.read_excel("미국개인소비자 구매량.xls", encoding='utf-8')
df_pce.head()
df_pce.shape

,observation_date,PCE
0,2014-01-01,11503.7
1,2014-02-01,11574.5
2,2014-03-01,11644.5
3,2014-04-01,11707.1
4,2014-05-01,11750.9


(82, 2)

In [34]:
from datetime import datetime
df_pce['월'] = '0'
for i in range(82):
    df_pce['월'][i] = str(df_pce['observation_date'][i])[:7]
df_pce.head()

,observation_date,PCE,월
0,2014-01-01,11503.7,2014-01
1,2014-02-01,11574.5,2014-02
2,2014-03-01,11644.5,2014-03
3,2014-04-01,11707.1,2014-04
4,2014-05-01,11750.9,2014-05


In [35]:
# PCE 데이터 병합
df_add_oil_pce = pd.merge(df_add_oil, df_pce, left_on="월", right_on="월", how="left")
df_add_oil_pce.head()

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,...,날짜,종가,오픈,고가,저가,거래량,변동 %,월,observation_date,PCE
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02,2014-02-01,11574.5
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,...,2014-02-18,102.43,100.32,103.25,100.23,177.37K,1.83%,2014-02,2014-02-01,11574.5
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02,2014-02-01,11574.5
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,...,2014-03-17,98.08,99.39,99.39,97.37,227.39K,-0.82%,2014-03,2014-03-01,11644.5
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,...,2014-04-01,99.74,101.53,101.57,99.28,237.56K,-1.81%,2014-04,2014-04-01,11707.1


### 북미선정량 지수

In [40]:
# 북미선정량 지수 불러오기
df_north = pd.read_excel("북미선적량 지수.xls", encoding='utf-8')
df_north.head()
df_north.shape

,observation_date,FRGSHPUSM649NCIS
0,2014-01-01,1.000
1,2014-02-01,1.073
2,2014-03-01,1.144
3,2014-04-01,1.161
4,2014-05-01,1.173


(82, 2)

In [42]:
df_north['월'] = '0'
for i in range(82):
    df_north['월'][i] = str(df_north['observation_date'][i])[:7]
df_north.head()

,observation_date,FRGSHPUSM649NCIS,월
0,2014-01-01,1.000,2014-01
1,2014-02-01,1.073,2014-02
2,2014-03-01,1.144,2014-03
3,2014-04-01,1.161,2014-04
4,2014-05-01,1.173,2014-05


In [43]:
# PCE 데이터 병합
df_add_oil_pce_north = pd.merge(df_add_oil_pce, df_north, left_on="월", right_on="월", how="left")
df_add_oil_pce_north.head()

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,...,오픈,고가,저가,거래량,변동 %,월,observation_date_x,PCE,observation_date_y,FRGSHPUSM649NCIS
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,...,NaN,NaN,NaN,NaN,NaN,2014-02,2014-02-01,11574.5,2014-02-01,1.073
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,...,100.32,103.25,100.23,177.37K,1.83%,2014-02,2014-02-01,11574.5,2014-02-01,1.073
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,...,NaN,NaN,NaN,NaN,NaN,2014-02,2014-02-01,11574.5,2014-02-01,1.073
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,...,99.39,99.39,97.37,227.39K,-0.82%,2014-03,2014-03-01,11644.5,2014-03-01,1.144
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,...,101.53,101.57,99.28,237.56K,-1.81%,2014-04,2014-04-01,11707.1,2014-04-01,1.161


In [44]:
df_add_oil_pce_north.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 609
Data columns (total 25 columns):
ONBOARD               610 non-null object
VESSEL                610 non-null object
POL                   610 non-null object
NATION                610 non-null object
DESTINATION           610 non-null object
20`                   610 non-null int64
RATE                  610 non-null float64
EX-RATE               610 non-null float64
O/FRT KRW             610 non-null int64
THC                   610 non-null int64
W/F                   610 non-null int64
D/FEE                 610 non-null int64
합계                    610 non-null int64
날짜                    469 non-null object
종가                    469 non-null float64
오픈                    469 non-null float64
고가                    469 non-null float64
저가                    469 non-null float64
거래량                   469 non-null object
변동 %                  469 non-null object
월                     610 non-null object
observation_d

In [47]:
final = df_add_oil_pce_north.drop(['날짜', '월', 'observation_date_x','observation_date_y'], axis=1, inplace=False)
final.head()

,ONBOARD,VESSEL,POL,NATION,DESTINATION,20`,RATE,EX-RATE,O/FRT KRW,THC,...,D/FEE,합계,종가,오픈,고가,저가,거래량,변동 %,PCE,FRGSHPUSM649NCIS
0,2014-02-09,WESTWOOD PACIFIC 022E,KRPUS,DET,USDET,1,2900.0,1085.6,3138960,115000,...,30000,3288310,NaN,NaN,NaN,NaN,NaN,NaN,11574.5,1.073
1,2014-02-18,WESTWOOD VICTORIA 069E,KRPUS,DET,USDET,1,2950.0,1075.1,3162990,115000,...,30000,3312340,102.43,100.32,103.25,100.23,177.37K,1.83%,11574.5,1.073
2,2014-02-23,WESTWOOD CASCADE 027E,KRPUS,DET,USDET,1,2900.0,1082.4,3154620,115000,...,30000,3303970,NaN,NaN,NaN,NaN,NaN,NaN,11574.5,1.073
3,2014-03-17,SEA-LAND INTREPID 1406,KRPUS,DET,USDET,1,3300.0,1077.8,3574890,115000,...,30000,3724240,98.08,99.39,99.39,97.37,227.39K,-0.82%,11644.5,1.144
4,2014-04-01,HYUNDAI TOKYO 071E,KRPUS,DET,USDET,1,3098.6,1076.5,3335643,115000,...,30000,3484993,99.74,101.53,101.57,99.28,237.56K,-1.81%,11707.1,1.161


In [48]:
final.isnull().sum()

ONBOARD               0
VESSEL                0
POL                   0
NATION                0
DESTINATION           0
20`                   0
RATE                  0
EX-RATE               0
O/FRT KRW             0
THC                   0
W/F                   0
D/FEE                 0
합계                    0
종가                  141
오픈                  141
고가                  141
저가                  141
거래량                 141
변동 %                141
PCE                   0
FRGSHPUSM649NCIS      0
dtype: int64

In [49]:
final = final.dropna()
final.shape

(469, 21)

In [50]:
final.to_csv('final_USDET.csv', index=False, encoding='cp949')